In [188]:
%pip install nbformat 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [54]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler as mm
from sklearn.preprocessing import StandardScaler as ss
from sklearn.model_selection import train_test_split

In [45]:
df = pd.read_excel (r'C:\Users\jwiggins\Downloads\January 2017 Crude Diet Yields and Product Stream Quality.xlsx',None)
sheets=pd.DataFrame(df.keys())
sheets_lower_underscore=sheets.applymap(lambda x: x.lower().replace(' ','_'))
sheets_lower_underscore

,0
0,vps2_crude_diet
1,vps2_yields
2,vps2_strmq
3,vps4_crude_diet
4,vps4_yields
5,vps4_strmq
6,vps5_crude_diet
7,vps5_yields
8,vps5_strmq


In [96]:
import pandas as pd

vps5_crude_diet = pd.read_excel (r'C:\Users\jwiggins\Downloads\January 2017 Crude Diet Yields and Product Stream Quality.xlsx',sheet_name='VPS5 Crude Diet', header=[5]).iloc[1:,:]
vps5_yields = pd.read_excel (r'C:\Users\jwiggins\Downloads\January 2017 Crude Diet Yields and Product Stream Quality.xlsx',sheet_name='VPS5 Yields', header=[5]).iloc[1:,:]
vps5_strmq = pd.read_excel (r'C:\Users\jwiggins\Downloads\January 2017 Crude Diet Yields and Product Stream Quality.xlsx',sheet_name='VPS5 StrmQ', header=[6]).iloc[1:,:]

all_data=pd.concat([vps5_crude_diet,vps5_yields,vps5_strmq],axis=1)

In [97]:
all_data.dtypes

Date        object
FP5FAPI    float64
FP5FSUL    float64
FP5FNIT    float64
FP5FV40    float64
            ...   
MNI        float64
MVA        float64
CON.4      float64
V50        float64
TAN.2      float64
Length: 148, dtype: object

In [177]:
def data_feed(csv):
    dropped_date = csv.select_dtypes(exclude=['datetime','object']).fillna(value=0).iloc[:30]
    data_input_scaled=ss().fit_transform(dropped_date)
    (x_train,x_test) = train_test_split(data_input_scaled,test_size=0.2,random_state=42)
    return x_train,x_test

In [178]:
data_feed(all_data)[0].dtype

dtype('float64')

In [179]:
def cnn_model(data_input):
    info=data_feed(data_input)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(info[0].shape[1],1)))
    model.add(tf.keras.layers.MaxPool1D(pool_size=2))
    model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(tf.keras.layers.MaxPool1D(pool_size=2))
    # model.add(tf.keras.layers.Dense(units=64, activation='relu', input_shape=(data_input_scaled.shape[1],)))
    # model.add(tf.keras.layers.Dense(units=16, activation='relu'))
    # model.add(tf.keras.layers.Dense(units=4, activation='relu'))
    # model.add(tf.keras.layers.Dense(units=1, activation='relu'))
    # model.add(tf.keras.layers.Dense(units=4, activation='relu'))
    # model.add(tf.keras.layers.Dense(units=16, activation='relu'))
    # model.add(tf.keras.layers.Dense(units=64, activation='relu'))




    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=info[0].shape[1], activation='linear'))
    model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae','accuracy'])
    es = tf.keras.callbacks.EarlyStopping(monitor='val_mae',patience=30)
    # checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath ="/content/sample_data")
    tbCallBack = tf.keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=1, write_graph=True, write_images=True)
    cb_list = [es,tbCallBack]
    model.fit(info[0], info[0], epochs=10000,validation_split=.1,shuffle=True,callbacks=cb_list)
    model.evaluate(info[0], info[0])
    return model


In [180]:
m=cnn_model(all_data)

m.summary()

Epoch 1/10000
1/1 [==============================] - 1s 534ms/step - loss: 1.0172 - mse: 1.0172 - mae: 0.6988 - accuracy: 0.0000e+00 - val_loss: 0.9472 - val_mse: 0.9472 - val_mae: 0.7126 - val_accuracy: 0.0000e+00
Epoch 2/10000
1/1 [==============================] - 0s 97ms/step - loss: 0.9610 - mse: 0.9610 - mae: 0.6837 - accuracy: 0.0000e+00 - val_loss: 0.9202 - val_mse: 0.9202 - val_mae: 0.7007 - val_accuracy: 0.0000e+00
Epoch 3/10000
1/1 [==============================] - 0s 89ms/step - loss: 0.9119 - mse: 0.9119 - mae: 0.6703 - accuracy: 0.0000e+00 - val_loss: 0.8965 - val_mse: 0.8965 - val_mae: 0.6894 - val_accuracy: 0.0000e+00
Epoch 4/10000
1/1 [==============================] - 0s 100ms/step - loss: 0.8677 - mse: 0.8677 - mae: 0.6578 - accuracy: 0.0000e+00 - val_loss: 0.8750 - val_mse: 0.8750 - val_mae: 0.6789 - val_accuracy: 0.0000e+00
Epoch 5/10000
1/1 [==============================] - 0s 99ms/step - loss: 0.8265 - mse: 0.8265 - mae: 0.6459 - accuracy: 0.0000e+00 - val_loss

In [167]:
 %load_ext tensorboard
%tensorboard --logdir Graph

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [168]:
  %reload_ext tensorboard


In [169]:
%tensorboard --logdir Graph

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [189]:
import matplotlib.pyplot as plt
data_mod=data_feed(all_data)
reconstructions = m.predict(data_mod[0])
train_loss = tf.keras.losses.mae(reconstructions, data_mod[0])

import plotly.express as px
import numpy as np
df_loss = pd.DataFrame(train_loss)
x=range(0,len(df_loss))
y=df_loss.iloc[:,0]
fig = px.line(x=x,y=y)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed